In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time
from sklearn.utils.class_weight import compute_class_weight
from transformers import EfficientNetForImageClassification
model=EfficientNetForImageClassification.from_pretrained('efficientnet-b0',num_classes=2)

In [2]:
DEVICE="cuda:0"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
# setAllSeeds(42)

In [3]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[0]
df = df[df["Category"]==category]

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


In [4]:
df.isna().sum()

id              0
Category        0
len             0
attr_1       4946
attr_2       5667
attr_3       5938
attr_4       5553
attr_5       6437
attr_6       5787
attr_7       5788
attr_8       5207
attr_9       6433
attr_10     11823
dtype: int64

In [5]:
i=5
# df.iloc[:,3:2+i]=np.nan
# df.iloc[:,3+i:]=np.nan
# df.iloc[:,[5,6]]=np.nan
print(df.head())


          id             Category  len     attr_1   attr_2   attr_3  \
51209  51375  Women Tops & Tunics   10      black  regular      NaN   
51210  51376  Women Tops & Tunics   10  navy blue   fitted     crop   
51211  51377  Women Tops & Tunics   10        red  regular  regular   
51212  51378  Women Tops & Tunics   10    default   fitted     crop   
51213  51379  Women Tops & Tunics   10    default     boxy  regular   

           attr_4  attr_5   attr_6      attr_7         attr_8  \
51209         NaN     NaN      NaN         NaN            NaN   
51210        high  casual  default       solid  short sleeves   
51211  round neck  casual  printed  typography     sleeveless   
51212    stylised  casual    solid       solid  short sleeves   
51213  round neck  casual  printed  typography  short sleeves   

                attr_9  attr_10  
51209  regular sleeves      NaN  
51210          default  knitted  
51211       sleeveless      NaN  
51212  regular sleeves  default  
51213       

In [7]:
delCol = []
idxCol = []
trackNum = []
weights=[]
m=nn.Softmax(dim=1)
l=[]
for i in range(1,11):
    uniName = df["attr_"+str(i)].dropna().unique()
    print(uniName)
    if(len(uniName)==0):
        delCol.append("attr_"+str(i))
    else:
        idxCol.append("attr_"+str(i))
        l.append('attr_'+str(i))
        trackNum.append(len(uniName))
        class_weights=compute_class_weight(class_weight="balanced",classes=uniName,y=df["attr_"+str(i)].dropna())
        weights.append(torch.tensor([1]*len(uniName),dtype=torch.float32).to(DEVICE))


['black' 'navy blue' 'red' 'default' 'maroon' 'white' 'green' 'blue'
 'pink' 'yellow' 'peach' 'multicolor']
['regular' 'fitted' 'boxy' 'default']
['crop' 'regular']
['high' 'round neck' 'stylised' 'sweetheart neck' 'v-neck' 'square neck'
 'default']
['casual' 'party']
['default' 'printed' 'solid']
['solid' 'typography' 'graphic' 'default' 'quirky' 'floral']
['short sleeves' 'sleeveless' 'three-quarter sleeves' 'long sleeves']
['regular sleeves' 'default' 'sleeveless' 'puff sleeves']
['knitted' 'default' 'ruffles' 'waist tie-ups' 'tie-ups' 'applique']


In [9]:
df = df.drop(delCol,axis=1)
df.shape

(19004, 13)

In [10]:
df.isna().sum()

id              0
Category        0
len             0
attr_1       4946
attr_2       5667
attr_3       5938
attr_4       5553
attr_5       6437
attr_6       5787
attr_7       5788
attr_8       5207
attr_9       6433
attr_10     11823
dtype: int64

In [11]:
# df.dropna(inplace=True)

In [ ]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].dropna().unique()
    # print(df.columns[i],labels)
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'black', 1: 'navy blue', 2: 'red', 3: 'default', 4: 'maroon', 5: 'white', 6: 'green', 7: 'blue', 8: 'pink', 9: 'yellow', 10: 'peach', 11: 'multicolor'}, 1: {0: 'regular', 1: 'fitted', 2: 'boxy', 3: 'default'}, 2: {0: 'crop', 1: 'regular'}, 3: {0: 'high', 1: 'round neck', 2: 'stylised', 3: 'sweetheart neck', 4: 'v-neck', 5: 'square neck', 6: 'default'}, 4: {0: 'casual', 1: 'party'}, 5: {0: 'default', 1: 'printed', 2: 'solid'}, 6: {0: 'solid', 1: 'typography', 2: 'graphic', 3: 'default', 4: 'quirky', 5: 'floral'}, 7: {0: 'short sleeves', 1: 'sleeveless', 2: 'three-quarter sleeves', 3: 'long sleeves'}, 8: {0: 'regular sleeves', 1: 'default', 2: 'sleeveless', 3: 'puff sleeves'}, 9: {0: 'knitted', 1: 'default', 2: 'ruffles', 3: 'waist tie-ups', 4: 'tie-ups', 5: 'applique'}}
{0: {'black': 0, 'navy blue': 1, 'red': 2, 'default': 3, 'maroon': 4, 'white': 5, 'green': 6, 'blue': 7, 'pink': 8, 'yellow': 9, 'peach': 10, 'multicolor': 11}, 1: {'regular': 0, 'fitted': 1, 'boxy': 2, 'default'

In [13]:
def categorize(example):
    for i in attrs:
        # print(example[attrs[i]],type(example[attrs[i]]),pd.isna(example[attrs[i]]))
        if not pd.isna(example[attrs[i]]):
            example[attrs[i]]=label2id[i][example[attrs[i]]]
        else:
            example[attrs[i]]=-100
    return example
df=df.apply(categorize,axis=1)
df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
51209,51375,Women Tops & Tunics,10,0,0,-100,-100,-100,-100,-100,-100,0,-100
51210,51376,Women Tops & Tunics,10,1,1,0,0,0,0,0,0,1,0
51211,51377,Women Tops & Tunics,10,2,0,1,1,0,1,1,1,2,-100
51212,51378,Women Tops & Tunics,10,3,1,0,2,0,2,0,0,0,1
51213,51379,Women Tops & Tunics,10,3,2,1,1,0,1,1,0,1,-100


In [14]:
df.isna().sum()

id          0
Category    0
len         0
attr_1      0
attr_2      0
attr_3      0
attr_4      0
attr_5      0
attr_6      0
attr_7      0
attr_8      0
attr_9      0
attr_10     0
dtype: int64

In [15]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# val_df,test_df=train_test_split(val_df,test_size=0.33)
# train_df=df
# val_df=train_df
# test_df=train_df

In [16]:
for i in l:
    print(train_df[i].value_counts())

attr_1
-100    3969
 0      2158
 5      1962
 3      1560
 8      1222
 7       753
 9       750
 4       635
 2       631
 6       537
 11      369
 1       341
 10      316
Name: count, dtype: int64
attr_2
-100    4546
 1      4406
 0      3841
 2      1656
 3       754
Name: count, dtype: int64
attr_3
 1      5863
-100    4747
 0      4593
Name: count, dtype: int64
attr_4
 1      5594
-100    4422
 0      1481
 5       946
 4       881
 6       865
 3       626
 2       388
Name: count, dtype: int64
attr_5
 0      9869
-100    5167
 1       167
Name: count, dtype: int64
attr_6
 2      6221
-100    4622
 1      3609
 0       751
Name: count, dtype: int64
attr_7
 0      6428
-100    4622
 3      1383
 1      1259
 4       663
 2       464
 5       384
Name: count, dtype: int64
attr_8
 0      5953
-100    4136
 3      2837
 2      1444
 1       833
Name: count, dtype: int64
attr_9
 0      5983
-100    5159
 3      2174
 1      1300
 2       587
Name: count, dtype: int64
attr_10
-100  

In [17]:
# for i in l:
#     print(i)
#     for j in set(train_df[i].unique()):
#         x=train_df[i][train_df[i]==j]
#         if x.count()<300 and j!=-100:
#             y=train_df[train_df[i]==j]
#             y.iloc[:,3:]=-100
#             y[i]=j
#             print(y)
#             train_df=pd.concat([train_df,y,y]).reset_index(drop=True)
# for i in l:
#     print(train_df[i].value_counts())

In [18]:
train_df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
56665,56831,Women Tops & Tunics,10,0,1,0,1,0,2,0,0,0,0
53783,53949,Women Tops & Tunics,10,-100,-100,-100,-100,0,-100,-100,0,1,-100
55647,55813,Women Tops & Tunics,10,-100,0,-100,0,0,2,0,2,1,-100
56911,57077,Women Tops & Tunics,10,1,0,1,2,0,2,0,0,1,-100
67440,67606,Women Tops & Tunics,10,11,0,1,4,0,1,5,3,0,3


In [19]:
from transformers import AutoImageProcessor
# model_name = 'google/vit-base-patch16-224'
model_name=f"vit4/{category}/final"
# model_name="google/vit-large-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name,use_fast=True)

In [20]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms =None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        attributes = self.fashionItems.iloc[idx, 3:]
        attributes = np.array(attributes)
        attributes = attributes.astype('float')
        
        

        
        inputs=processor(image, return_tensors='pt')
        inputs['labels']=torch.tensor(attributes, dtype=torch.long)
        if self.transforms:
            inputs['pixel_values'] = self.transforms(inputs['pixel_values'])
        

        return inputs


In [21]:
from torchvision import transforms
transformations=transforms.Compose([
    transforms.RandomRotation(degrees=90),
    transforms.GaussianBlur(kernel_size=(9, 9), sigma=(0.1, 2.0)),
])
transformations=None
train_fashion_data = CustomFashionManager(csv_file=train_df,
                                    root_dir='train_images',transforms=transformations)
val_fashion_data = CustomFashionManager(csv_file=val_df,
                                    root_dir='train_images')
# test_fashion_data = CustomFashionManager(csv_file=test_df,root_dir='train_images')
        

In [22]:
import sys
from typing import List
from transformers import (
    ViTConfig,ViTPreTrainedModel,ViTForImageClassification,AutoConfig,ResNetConfig,ResNetModel,ResNetPreTrainedModel,ResNetForImageClassification)


class CustomConfig(ViTConfig):
    def __init__(self,num_classes_per_label:List[int]=[1],**kwargs):
        super().__init__(**kwargs)
        self.num_classes_per_label = num_classes_per_label
        self.attn_implementation="sdpa"
        self.hidden_size_2 = 382
        self.attention_probs_dropout_prob=0.1
        self.hidden_dropout_prob=0.1

            
class MultiLabelMultiClassViT(ViTPreTrainedModel):
    config_class=CustomConfig
    def __init__(self, config: CustomConfig) -> None:
        super().__init__(config)

        self.vit = ViTModel(config, add_pooling_layer=True)
        self.classifiers = nn.ModuleList([
            # nn.Sequential(
            #     nn.Linear(config.hidden_size, config.hidden_size_2),
            #     nn.GELU(),
            #     nn.Linear(config.hidden_size_2, num_classes),
            #     nn.Dropout(0.2)
            # )
            nn.Sequential(
            nn.Linear(config.hidden_size, num_classes),
            nn.Dropout(0.25)) 
            # nn.Linear(config.hidden_size, num_classes)
            for num_classes in config.num_classes_per_label
        ])
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, pixel_values,labels=None):
        # outputs = self.vit(pixel_values).last_hidden_state[:, 0, :]  # CLS token representation
        outputs=self.vit(pixel_values).pooler_output
        logits = [classifier(outputs) for classifier in self.classifiers]
        if labels is not None:
            loss=0
            for i in range(len(logits)):
                target=labels[:,i]
                loss += torch.nn.functional.cross_entropy(logits[i], target,ignore_index=-100,weight=weights[i])
            # loss=loss/len(logits)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

    
# Example usage
num_labels = len(trackNum)  # For example, 5 different labels



In [23]:
config=ViTConfig.from_pretrained(model_name)
config=CustomConfig(num_classes_per_label=trackNum,**config.to_dict())
model = MultiLabelMultiClassViT.from_pretrained(model_name, config=config)

# model=MultiLabelMultiClassViT.from_pretrained(model_name)
# for param in model.vit.parameters():
#     param.requires_grad = False
# model.vit.pooler.parameters().__next__().requires_grad=True
print(summary(model))

Layer (type:depth-idx)                                       Param #
MultiLabelMultiClassViT                                      --
├─ViTModel: 1-1                                              --
│    └─ViTEmbeddings: 2-1                                    152,064
│    │    └─ViTPatchEmbeddings: 3-1                          590,592
│    │    └─Dropout: 3-2                                     --
│    └─ViTEncoder: 2-2                                       --
│    │    └─ModuleList: 3-3                                  85,054,464
│    └─LayerNorm: 2-3                                        1,536
│    └─ViTPooler: 2-4                                        --
│    │    └─Linear: 3-4                                      590,592
│    │    └─Tanh: 3-5                                        --
├─ModuleList: 1-2                                            --
│    └─Sequential: 2-5                                       --
│    │    └─Linear: 3-6                                      9,228
│    │

In [24]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report
batch_size = 32
def collate_fn(batch):
    return {
        'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
        'labels': torch.stack([x['labels'] for x in batch])
    }

def compute_metrics(pred):
    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    f1s=[]
    accs=[]
    for i in range(labels.shape[1]):
        non_padding_indices = [j for j, label in enumerate(labels[:,i]) if label != -100]
        labels_ = [labels[j,i] for j in non_padding_indices]
        probs_ = [probs[j,i] for j in non_padding_indices]
        report=classification_report(labels_,probs_,output_dict=True)
        # print(classification_report(labels_,probs_))
        f1s.append(report['macro avg']['f1-score'])
        accs.append(report['accuracy'])
    metrics=pd.DataFrame({"attr":list(attrs.values()),"acc":accs,"f1":f1s})
    print(metrics)

    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    labels=labels.flatten()
    probs=probs.flatten()
    non_padding_indices = [i for i, label in enumerate(labels) if label != -100]
    labels = [labels[i] for i in non_padding_indices]
    probs = [probs[i] for i in non_padding_indices]
    print(classification_report(labels,probs))
    return {'accuracy': np.mean(accs),"macro avg f1":np.mean(f1s)}

    # report=classification_report(labels,probs,output_dict=True)
    # return {'accuracy': report['accuracy'],"macro avg f1":report['macro avg']['f1-score']}

training_args = TrainingArguments(
  output_dir="./vit4/"+category,
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=10,
  bf16=True,
  learning_rate=2e-4,
#   dataloader_num_workers=20,lr
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  metric_for_best_model="macro avg f1"
  
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    eval_dataset=val_fashion_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()
trainer.save_model(f"./vit4/{category}/final")

[2024-11-06 19:00:32,350] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [26]:
trainer.evaluate(val_fashion_data)
#1-54(imbalance),2-65,3-85,5-56,6-70,7-52(imbalance),8-51(imbalance),9-60(imbalance),10-49(imbalance)
#4-30,

[2024-11-06 18:04:02,377] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


              precision    recall  f1-score   support

           0       0.93      0.93      0.93     10998
           1       0.89      0.89      0.89      6026
           2       0.90      0.90      0.90      2981
           3       0.87      0.86      0.86      2473
           4       0.71      0.72      0.72       658
           5       0.87      0.86      0.87       938
           6       0.74      0.70      0.72       346
           7       0.73      0.81      0.77       192
           8       0.85      0.85      0.85       310
           9       0.80      0.85      0.82       171
          10       0.82      0.74      0.78        81
          11       0.61      0.49      0.54        87

    accuracy                           0.90     25261
   macro avg       0.81      0.80      0.80     25261
weighted avg       0.90      0.90      0.90     25261



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


{'eval_loss': 4.760494709014893,
 'eval_model_preparation_time': 0.0051,
 'eval_accuracy': 0.8960452872016151,
 'eval_macro avg f1': 0.8041487151375172,
 'eval_runtime': 20.0533,
 'eval_samples_per_second': 189.545,
 'eval_steps_per_second': 1.496}

In [ ]:
from transformers import pipeline,AutoModelForCausalLM,LlamaForCausalLM

model=AutoModelForCausalLM.from_pretrained(f"Llama3")
model.generate()